<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
!pip install pandas
m = Mystem()
import re
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install swifter
import swifter



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Перед нами данные о почти 160.000 комментариях, размеченные по  степени токсичности

In [4]:
data['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

Видно, что отсутствует баланс. При классификации попробуем сбалансировать

In [5]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

data['text'] = data['text'].apply(cleaning)

In [6]:
data['text'].head()

0    explanation why the edits made under my userna...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3    more i cant make any real suggestions on impro...
4    you sir are my hero any chance you remember wh...
Name: text, dtype: object

Приведем данные в порядок - уберем лишние символы и сделаем текст с маленькой буквы

In [7]:
%%time
lmtzr = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    #text=text.split()
    lemmatized_output = ' '.join(lmtzr.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text))
    return lemmatized_output
                                  



#lemmatize('this is an ambulance calls michaels maggots nuggets feet mice')
data['lemm_text'] = data['text'].swifter.apply(lemmatize)


# data = data.drop(['text'], axis=1)

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 20min 57s, sys: 1min 59s, total: 22min 57s
Wall time: 23min 31s


Лемматизируем

Эта ячейка достаточно долго запускалась, особенно локально. Как с этим бороться? Можно ли как то ускорить процесс?

In [8]:
stopwords = set(nltk_stopwords.words('english'))

In [9]:
target=data['toxic']
features=data['lemm_text']


train_features,test_features,train_target,test_target=train_test_split(features,target,random_state=12345,test_size=.25)

Поделим на две выборки, в соотношении 3:1

In [10]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_train = count_tf_idf.fit_transform(train_features)
tf_test = count_tf_idf.transform(test_features)


Прекратим данные в вектор для дальнейшей обработки.

## Обучение

Посмотрим на логистическую регрессию

In [11]:
param_grid = [{'C': list(range(1,14,3)),'class_weight':[None,'balanced']}]
model=LogisticRegression(random_state=12345,max_iter=450)
grid_cv = GridSearchCV(model, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)


In [12]:
%%time
best_grid = grid_cv.fit(tf_train, train_target)
print('Best parameters:', grid_cv.best_params_)
print('Best score:', grid_cv.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .............................C=1, class_weight=None; total time=  54.4s
[CV] END .............................C=1, class_weight=None; total time=  53.7s
[CV] END .............................C=1, class_weight=None; total time=  56.3s
[CV] END .........................C=1, class_weight=balanced; total time= 1.1min
[CV] END .........................C=1, class_weight=balanced; total time=  27.1s
[CV] END .........................C=1, class_weight=balanced; total time=  31.5s
[CV] END .............................C=4, class_weight=None; total time= 1.5min
[CV] END .............................C=4, class_weight=None; total time= 1.2min
[CV] END .............................C=4, class_weight=None; total time= 1.2min
[CV] END .........................C=4, class_weight=balanced; total time= 1.2min
[CV] END .........................C=4, class_weight=balanced; total time= 1.9min
[CV] END .........................C=4, class_wei

Теперь на случайный лес

In [13]:
params_forest = {
    'n_estimators': list(range(50,151,50)),
    'max_depth':[5,10],
    'max_features' : list(range(1,15, 2))}


model_forest = RandomForestClassifier(random_state=12345)
                                 
grid = GridSearchCV(model_forest, param_grid=params_forest, scoring='f1', cv=3, verbose=2, n_jobs=-1)

In [14]:
%%time

best_grid = grid.fit(tf_train, train_target)
print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] END .......max_depth=5, max_features=1, n_estimators=50; total time=   1.1s
[CV] END .......max_depth=5, max_features=1, n_estimators=50; total time=   1.1s
[CV] END .......max_depth=5, max_features=1, n_estimators=50; total time=   1.2s
[CV] END ......max_depth=5, max_features=1, n_estimators=100; total time=   2.0s
[CV] END ......max_depth=5, max_features=1, n_estimators=100; total time=   2.0s
[CV] END ......max_depth=5, max_features=1, n_estimators=100; total time=   2.1s
[CV] END ......max_depth=5, max_features=1, n_estimators=150; total time=   3.1s
[CV] END ......max_depth=5, max_features=1, n_estimators=150; total time=   3.2s
[CV] END ......max_depth=5, max_features=1, n_estimators=150; total time=   3.1s
[CV] END .......max_depth=5, max_features=3, n_estimators=50; total time=   1.2s
[CV] END .......max_depth=5, max_features=3, n_estimators=50; total time=   1.2s
[CV] END .......max_depth=5, max_features=3, n_

Вероятно, случайный лес вообще не подходит для классификации. Не будем его использовать.

## Выводы

In [15]:
model = LogisticRegression(random_state=12345, C = 10, class_weight= 'balanced', max_iter=400)
model.fit(tf_train, train_target)
pred = model.predict(tf_test)
f1_score(test_target, pred)

0.7649685174585005

**ВЫВОД**: таким образом, нам удалось выявить модель Логистической Регрессии для классификации токсичных комментариев, которая показала метрику F-1 Score выше, чем 0.75.

**ИТОГИ ПРОЕКТА:**

* данные импротированы, проверены на пропуски

* проверили баланс классов, лемматизировали и токенизировали текст для обработки

* разделили данные и векторизовали строки

* рассмотрели модели логистической регрессии и случайного леса

* **получили модель логистической регрессии с f1-score на уровне 0.77**

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны